# Exploring the Structure of the Data

### Import

In [1]:
pwd()

u'/home/lachlan'

In [2]:
df = spark.read.json('/twitter/movie/DeerAntMan/*.gz')

In [ ]:
#df.printSchema()

### Getting the Number of rows

In [3]:
df.count()

192718

In [ ]:
df.columns

In [ ]:
df.select('actor').first()

In [ ]:
df.select('body').first()

In [ ]:
df.select('gnip').first()

In [ ]:
df.select('postedTime').first()

In [4]:
small_df = df.select('body', 'postedTime', 'retweetCount').na.drop()

In [5]:
small_df.count()

181507

## Formatting the date string using DateUtil (deprecated)

In [6]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [7]:
from dateutil.parser import parse as parse_date

In [22]:
def parsedate(dateString):
    try:
        return parse_date(dateString).date()
    except:
        pass

In [23]:
parsedate_udf = udf(parsedate)

In [24]:
small_df2 = small_df.withColumn('date', parsedate_udf(col('postedTime')))

In [25]:
small_df2.printSchema()

root
 |-- body: string (nullable = true)
 |-- postedTime: string (nullable = true)
 |-- retweetCount: long (nullable = true)
 |-- date: string (nullable = true)



In [26]:
small_df2.show(5)

+--------------------+--------------------+------------+--------------------+
|                body|          postedTime|retweetCount|                date|
+--------------------+--------------------+------------+--------------------+
|RT @Kotaku: The n...|2015-04-13T13:30:...|          11|java.util.Gregori...|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         428|java.util.Gregori...|
|RT @SuperheroFeed...|2015-04-13T13:30:...|         426|java.util.Gregori...|
|Second Action-Pac...|2015-04-13T13:30:...|           0|java.util.Gregori...|
|RT @screenrant: '...|2015-04-13T13:30:...|           8|java.util.Gregori...|
+--------------------+--------------------+------------+--------------------+
only showing top 5 rows



ImportError: No module named spark.sql.types.DateType

## Formatting the Date String with PySpark inbuilt

In [31]:
small_df2 = small_df.withColumn('dateColumn', small_df['postedTime'].cast('date'))

In [32]:
small_df2.printSchema()

root
 |-- body: string (nullable = true)
 |-- postedTime: string (nullable = true)
 |-- retweetCount: long (nullable = true)
 |-- dateColumn: date (nullable = true)



In [33]:
tweetsByDay = small_df2.groupby(small_df2.dateColumn).count()

In [37]:
tweetsByDay.describe().show()

+-------+------------------+
|summary|             count|
+-------+------------------+
|  count|               370|
|   mean|490.55945945945945|
| stddev|1122.6372130715913|
|    min|                 5|
|    max|             10942|
+-------+------------------+

